<a href="https://colab.research.google.com/github/zami-dot/Data_Science/blob/main/LAB_9_20SW046.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

LAB 9

Roll_no 20SW046

Section 2

In [1]:
import sqlite3

conn = sqlite3.connect('worldbank.db')
cur = conn.cursor()
cur.execute("DROP TABLE IF EXISTS gdp")
cur.execute("CREATE TABLE gdp (countryname TEXT, countrycode TEXT, year INTEGER, gdp REAL, PRIMARY KEY (countrycode, year));")
conn.commit()
conn.close()

In [2]:

def extract_lines(file):
    while True:
        line = file.readline()
        if not line:
            break
        yield line

In [3]:

import pandas as pd
import numpy as np
import sqlite3

def transform_indicator_data(data, colnames):
    # Create a list to store the results
    results = []

    countryname = data[0].replace('"', '')
    countrycode = data[1].replace('"', '')

    for i in range(2, len(data)):
        year = colnames[i]
        gdp_str = data[i].replace('"', '')

        if gdp_str.strip() != '' and gdp_str.replace(',', '', 1).replace('.', '', 1).isdigit():
            gdp = float(gdp_str)
            results.append([countryname, countrycode, int(year), gdp])

    return results

In [4]:
def load_indicator_data(results):
    # Connect to the worldbank.db database using the sqlite3 library
    conn = sqlite3.connect('worldbank.db')

    cur = conn.cursor()

    if results:
        for result in results:
            countryname, countrycode, year, gdp = result

            sql_string = f"INSERT INTO gdp (countryname, countrycode, year, gdp) VALUES (?, ?, ?, ?);"

            try:
                cur.execute(sql_string, (countryname, countrycode, year, gdp))
            except Exception as e:
                print('Error occurred:', e, result)

    conn.commit()
    conn.close()


In [6]:
with open('/content/gdp_data.csv') as f:
    for line in extract_lines(f):
        data = line.split(',')
        if len(data) == 63:
            if data[0] == '"Country Name"':
                colnames = []
                for i, datum in enumerate(data):
                    colnames.append(datum.replace('"', ''))
            else:
                results = transform_indicator_data(data, colnames)
                load_indicator_data(results)

In [7]:

conn = sqlite3.connect('worldbank.db')

cur = conn.cursor()

df = pd.read_sql("SELECT * FROM gdp", con=conn)

conn.commit()
conn.close()

df

,countryname,countrycode,year,gdp
0,Aruba,ABW,1994,1.330168e+09
1,Aruba,ABW,1995,1.320670e+09
2,Aruba,ABW,1996,1.379888e+09
3,Aruba,ABW,1997,1.531844e+09
4,Aruba,ABW,1998,1.665363e+09
...,...,...,...,...
10955,Zimbabwe,ZWE,2013,1.545177e+10
10956,Zimbabwe,ZWE,2014,1.589105e+10
10957,Zimbabwe,ZWE,2015,1.630467e+10
10958,Zimbabwe,ZWE,2016,1.661996e+10
